In [1]:
import os
import re
from openai import OpenAI
from moviepy.editor import VideoFileClip, AudioFileClip, concatenate_videoclips
from moviepy.video.fx.all import speedx

# Initialize OpenAI client
client = OpenAI()

def process_srt_file(srt_file_path):
    """Process the SRT file and create audio files from the subtitles."""
    with open(srt_file_path, "r") as srt_file:
        srt_content = srt_file.read()

    subtitle_pattern = re.compile(r"(\d+)\n(\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3})\n(.+?)\n\n", re.DOTALL)
    subtitles = subtitle_pattern.findall(srt_content)

    output_folder = "subAudio"
    os.makedirs(output_folder, exist_ok=True)

    # Clear output folder
    for file_name in os.listdir(output_folder):
        os.remove(os.path.join(output_folder, file_name))

    for idx, subtitle_block in enumerate(subtitles, start=1):
        subtitle_text = subtitle_block[2].strip()
        print(f"Processing subtitle {idx}: {subtitle_text}")
        response = client.audio.speech.create(model="tts-1", voice="alloy", input=subtitle_text)
        output_filename = os.path.join(output_folder, f"subtitle_block_{idx}.mp3")
        response.stream_to_file(output_filename)
        print(f"Subtitle block {idx} saved as {output_filename}")

def parse_srt_file(srt_file_path):
    """Parse the timecodes and texts from the SRT file."""
    with open(srt_file_path, "r") as srt_file:
        srt_content = srt_file.read()

    subtitle_pattern = re.compile(r"(\d+)\n(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})\n(.+?)\n\n", re.DOTALL)
    return subtitle_pattern.findall(srt_content)

def convert_timecode_to_seconds(timecode):
    """Convert SRT timecode format (HH:MM:SS,ms) to seconds."""
    hours, minutes, seconds = timecode.split(":")
    seconds, milliseconds = seconds.split(",")
    return int(hours) * 3600 + int(minutes) * 60 + int(seconds) + int(milliseconds) / 1000

def cut_video_with_moviepy(video_file_path, srt_file_path):
    """Cut the video into parts based on the SRT timecodes using moviepy."""
    subtitles = parse_srt_file(srt_file_path)
    output_folder = "video_clips"
    os.makedirs(output_folder, exist_ok=True)

    # Clear output folder
    for file_name in os.listdir(output_folder):
        os.remove(os.path.join(output_folder, file_name))

    video = VideoFileClip(video_file_path)

    for idx, (_, start_time, end_time, _) in enumerate(subtitles, start=1):
        start_seconds = convert_timecode_to_seconds(start_time)
        end_seconds = convert_timecode_to_seconds(end_time)

        video_clip = video.subclip(start_seconds, end_seconds)
        output_filename = os.path.join(output_folder, f"video_clip_{idx}.mp4")

        video_clip.write_videofile(output_filename, codec="libx264", logger=None)
        print(f"Clip {idx} saved: {output_filename}")

def sync_audio_video():
    """Sync audio and video by adjusting video speed."""
    video_dir, audio_dir, output_dir = 'video_clips', 'subAudio', 'truncated_videos'
    os.makedirs(output_dir, exist_ok=True)

    # Clear output folder
    for file_name in os.listdir(output_dir):
        os.remove(os.path.join(output_dir, file_name))

    video_files = sorted([f for f in os.listdir(video_dir) if f.endswith('.mp4')])
    audio_files = sorted([f for f in os.listdir(audio_dir) if f.endswith('.mp3')])

    for video_file, audio_file in zip(video_files, audio_files):
        video_path = os.path.join(video_dir, video_file)
        audio_path = os.path.join(audio_dir, audio_file)

        video_clip = VideoFileClip(video_path)
        audio_clip = AudioFileClip(audio_path)

        speed_factor = video_clip.duration / audio_clip.duration
        adjusted_video = speedx(video_clip, factor=speed_factor)

        final_clip = adjusted_video.set_audio(audio_clip)
        output_path = os.path.join(output_dir, f'truncated_{video_file}')

        final_clip.write_videofile(output_path, codec='libx264', audio_codec='aac', logger=None)

def combine_audio_video():
    """Combine individual audio and video clips into a final video."""
    audio_folder, video_folder, output_folder = 'subAudio', 'truncated_videos', 'output_clips'
    os.makedirs(output_folder, exist_ok=True)

    # Clear output folder
    for file_name in os.listdir(output_folder):
        os.remove(os.path.join(output_folder, file_name))

    audio_files = sorted([f for f in os.listdir(audio_folder) if f.endswith('.mp3')])
    video_files = sorted([f for f in os.listdir(video_folder) if f.endswith('.mp4')])

    for audio_file, video_file in zip(audio_files, video_files):
        audio_path = os.path.join(audio_folder, audio_file)
        video_path = os.path.join(video_folder, video_file)
        output_path = os.path.join(output_folder, f'combined_{video_file}')

        video_with_audio = VideoFileClip(video_path).set_audio(AudioFileClip(audio_path))
        video_with_audio.write_videofile(output_path, codec='libx264', logger=None)
        print(f'Combined {audio_file} with {video_file} into {output_path}')

def merge_all_clips():
    """Merge all video clips into one final video."""
    folder_path = 'output_clips'
    file_list = sorted([f for f in os.listdir(folder_path) if f.endswith('.mp4')], key=lambda f: int(re.search(r'(\d+)', f).group(0)))

    clips = [VideoFileClip(os.path.join(folder_path, filename)) for filename in file_list]
    final_clip = concatenate_videoclips(clips, method="compose")
    final_clip.write_videofile('output_combined.mp4', codec='libx264', logger=None)

# Execute the functions in the desired sequence
srt_file_path = "./descriptive_ai/output_subtitles.srt"
video_file_path = "./descriptive_ai/videos/input.mp4"

process_srt_file(srt_file_path)


Processing subtitle 1: A girl sits on a bed with a laptop, rain hits a window. Soft music plays in the background.


C:\Users\compu\AppData\Local\Temp\ipykernel_31824\4146906734.py:32: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(output_filename)


Subtitle block 1 saved as subAudio\subtitle_block_1.mp3
Processing subtitle 2: The girl glances towards the window, surprised by the sudden thunder. Rain pours heavily outside.
Subtitle block 2 saved as subAudio\subtitle_block_2.mp3
Processing subtitle 3: She stands up, with a slight frown, and begins to walk towards the window, her teddy bear in her arms.
Subtitle block 3 saved as subAudio\subtitle_block_3.mp3
Processing subtitle 4: The girl picks up a bright yellow raincoat, joyfully putting it on while she prepares to go outside.
Subtitle block 4 saved as subAudio\subtitle_block_4.mp3
Processing subtitle 5: She stands at the door, stepping outside into the pouring rain, smiling brightly.
Subtitle block 5 saved as subAudio\subtitle_block_5.mp3


In [2]:
cut_video_with_moviepy(video_file_path, srt_file_path)


Moviepy - Building video video_clips\video_clip_1.mp4.
MoviePy - Writing audio in video_clip_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\video_clip_1.mp4



Moviepy - Done !
Moviepy - video ready video_clips\video_clip_1.mp4
Clip 1 saved: video_clips\video_clip_1.mp4
Moviepy - Building video video_clips\video_clip_2.mp4.
MoviePy - Writing audio in video_clip_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\video_clip_2.mp4



Moviepy - Done !
Moviepy - video ready video_clips\video_clip_2.mp4
Clip 2 saved: video_clips\video_clip_2.mp4
Moviepy - Building video video_clips\video_clip_3.mp4.
MoviePy - Writing audio in video_clip_3TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready video_clips\video_clip_3.mp4
Clip 3 saved: video_clips\video_clip_3.mp4
Moviepy - Building video video_clips\video_clip_4.mp4.
MoviePy - Writing audio in video_clip_4TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\video_clip_4.mp4



Moviepy - Done !
Moviepy - video ready video_clips\video_clip_4.mp4
Clip 4 saved: video_clips\video_clip_4.mp4
Moviepy - Building video video_clips\video_clip_5.mp4.
MoviePy - Writing audio in video_clip_5TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video_clips\video_clip_5.mp4



Moviepy - Done !
Moviepy - video ready video_clips\video_clip_5.mp4
Clip 5 saved: video_clips\video_clip_5.mp4


In [3]:
sync_audio_video()


Moviepy - Building video truncated_videos\truncated_video_clip_1.mp4.
MoviePy - Writing audio in truncated_video_clip_1TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video truncated_videos\truncated_video_clip_1.mp4



Moviepy - Done !
Moviepy - video ready truncated_videos\truncated_video_clip_1.mp4
Moviepy - Building video truncated_videos\truncated_video_clip_2.mp4.
MoviePy - Writing audio in truncated_video_clip_2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video truncated_videos\truncated_video_clip_2.mp4



Moviepy - Done !
Moviepy - video ready truncated_videos\truncated_video_clip_2.mp4
Moviepy - Building video truncated_videos\truncated_video_clip_3.mp4.
MoviePy - Writing audio in truncated_video_clip_3TEMP_MPY_wvf_snd.mp4


...

Moviepy - Done !
Moviepy - video ready truncated_videos\truncated_video_clip_3.mp4
Moviepy - Building video truncated_videos\truncated_video_clip_4.mp4.
MoviePy - Writing audio in truncated_video_clip_4TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video truncated_videos\truncated_video_clip_4.mp4



Moviepy - Done !
Moviepy - video ready truncated_videos\truncated_video_clip_4.mp4
Moviepy - Building video truncated_videos\truncated_video_clip_5.mp4.
MoviePy - Writing audio in truncated_video_clip_5TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video truncated_videos\truncated_video_clip_5.mp4



Moviepy - Done !
Moviepy - video ready truncated_videos\truncated_video_clip_5.mp4


In [4]:
combine_audio_video()


Moviepy - Building video output_clips\combined_truncated_video_clip_1.mp4.
MoviePy - Writing audio in combined_truncated_video_clip_1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_clips\combined_truncated_video_clip_1.mp4



Moviepy - Done !
Moviepy - video ready output_clips\combined_truncated_video_clip_1.mp4
Combined subtitle_block_1.mp3 with truncated_video_clip_1.mp4 into output_clips\combined_truncated_video_clip_1.mp4
Moviepy - Building video output_clips\combined_truncated_video_clip_2.mp4.
MoviePy - Writing audio in combined_truncated_video_clip_2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_clips\combined_truncated_video_clip_2.mp4



Moviepy - Done !
Moviepy - video ready output_clips\combined_truncated_video_clip_2.mp4
Combined subtitle_block_2.mp3 with truncated_video_clip_2.mp4 into output_clips\combined_truncated_video_clip_2.mp4
Moviepy - Building video output_clips\combined_truncated_video_clip_3.mp4.
MoviePy - Writing audio in combined_truncated_video_clip_3TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready output_clips\combined_truncated_video_clip_3.mp4
Combined subtitle_block_3.mp3 with truncated_video_clip_3.mp4 into output_clips\combined_truncated_video_clip_3.mp4
Moviepy - Building video output_clips\combined_truncated_video_clip_4.mp4.
MoviePy - Writing audio in combined_truncated_video_clip_4TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_clips\combined_truncated_video_clip_4.mp4



Moviepy - Done !
Moviepy - video ready output_clips\combined_truncated_video_clip_4.mp4
Combined subtitle_block_4.mp3 with truncated_video_clip_4.mp4 into output_clips\combined_truncated_video_clip_4.mp4
Moviepy - Building video output_clips\combined_truncated_video_clip_5.mp4.
MoviePy - Writing audio in combined_truncated_video_clip_5TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_clips\combined_truncated_video_clip_5.mp4



Moviepy - Done !
Moviepy - video ready output_clips\combined_truncated_video_clip_5.mp4
Combined subtitle_block_5.mp3 with truncated_video_clip_5.mp4 into output_clips\combined_truncated_video_clip_5.mp4


In [5]:
merge_all_clips()

Moviepy - Building video output_combined.mp4.
MoviePy - Writing audio in output_combinedTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output_combined.mp4



Moviepy - Done !
Moviepy - video ready output_combined.mp4
